# Predictive Analytics : SVM
**Author: Onur Koc**

**Task:**

Develop two prediction models that predict taxi trip demand using a) support vector machines and b) neural networks (deep learning) in spatio-temporal resolution (i.e., spatial-unit and time buckets). In other words, your method should predict for each spatial unit (hexagon and census tract) and time-basket (e.g., 08am-11.59am) the taxi demand. Also adevise a reasonable validation strategy for your prediction model (i.e., definition of test, training data etc).

## Preamble

In [3]:
# importing required modules 
import numpy as np
import pandas as pd
import matplotlib as mpl
import h3
import h3pandas
import geopandas as gpd
import matplotlib.pyplot as plt
from datetime import datetime
import holidays

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.pipeline import make_pipeline

%matplotlib inline

**Datasets**

**Summary of Multiindex Datasets:**

<ins>Spatio-Temporal Aggregations for Census Tracts:</ins>
+ taxi_by_census_tract_1H &nbsp;&nbsp;: Hourly Aggregation
+ taxi_by_census_tract_2H &nbsp;&nbsp;: 2 Hourly Aggregation
+ taxi_by_census_tract_6H &nbsp;&nbsp;: 6 Hourly Aggregation
+ taxi_by_census_tract_24H &nbsp;&nbsp;: Daily Aggregation

<ins>Spatio-Temporal Aggregations for Hexagons with Resolution 6:</ins>
+ taxi_by_h3_6_1H &nbsp;&nbsp;: Hourly Aggregation
+ taxi_by_h3_6_2H &nbsp;&nbsp;: 2 Hourly Aggregation
+ taxi_by_h3_6_6H &nbsp;&nbsp;: 6 Hourly Aggregation
+ taxi_by_h3_6_24H &nbsp;&nbsp;: Daily Aggregation

<ins>Spatio-Temporal Aggregations for Hexagons with Resolution 7:</ins>
+ taxi_by_h3_7_1H &nbsp;&nbsp;: Hourly Aggregation
+ taxi_by_h3_7_2H &nbsp;&nbsp;: 2 Hourly Aggregation
+ taxi_by_h3_7_6H &nbsp;&nbsp;: 6 Hourly Aggregation
+ taxi_by_h3_7_24H &nbsp;&nbsp;: Daily Aggregation

**Summary of Features**

<ins>Target Variable:</ins> 
+ trip_demand : number of starting trips for each time bucket of year 2021

<ins>Features:</ins>

+ precip : binary feature that indicates whether the current time-bucket has precipitation or not.
+ windspeed_z : normalized average windspeed
+ temp_z : normalized average temperature
+ temp_z_ma_7d : moving average of the normalized temperature over 7 days
+ temp_z_std_7d : moving standard deviation of the normalized temperature over 7 days
+ weekday_sin : sine tranformed weekday
+ weekday_cos : cosine transformed weekday
+ is_weekend : binary feature to indicate whether the current time-bucket falls on a weekend or not
+ is_holiday : binary feature to indicate whether the current time-bucket falls on a holiday or not
+ 4 season dummies : season_Autumn, season_Spring, season_Summer & season_Winter

**Reading the Datasets**

In [4]:
file_path = "./data/datasets/"

taxi_by_h3_6_1H = pd.read_pickle(f"{file_path}taxi_by_h3_6_1H.pkl")
taxi_by_h3_6_2H = pd.read_pickle(f"{file_path}taxi_by_h3_6_2H.pkl")
taxi_by_h3_6_6H = pd.read_pickle(f"{file_path}taxi_by_h3_6_6H.pkl")
taxi_by_h3_6_24H = pd.read_pickle(f"{file_path}taxi_by_h3_6_24H.pkl")

taxi_by_h3_7_1H = pd.read_pickle(f"{file_path}taxi_by_h3_7_1H.pkl")
taxi_by_h3_7_2H = pd.read_pickle(f"{file_path}taxi_by_h3_7_2H.pkl")
taxi_by_h3_7_6H = pd.read_pickle(f"{file_path}taxi_by_h3_7_6H.pkl")
taxi_by_h3_7_24H = pd.read_pickle(f"{file_path}taxi_by_h3_7_24H.pkl")

taxi_by_census_tract_1H = pd.read_pickle(f"{file_path}taxi_by_census_tract_1H.pkl")
taxi_by_census_tract_2H = pd.read_pickle(f"{file_path}taxi_by_census_tract_2H.pkl")
taxi_by_census_tract_6H = pd.read_pickle(f"{file_path}taxi_by_census_tract_6H.pkl")
taxi_by_census_tract_24H = pd.read_pickle(f"{file_path}taxi_by_census_tract_24H.pkl")

## Optimizing Hyperparameters with Randomized Search

+ **We are using time series CV split with 5 folds**
+ **I would not advise to run the Randomized Search, as it takes a lot of time**

**Given our current computational capabilities, we have developed a method to approximate the best parameter set.**
**We know the top 5 demand areas for all spatial resolutions from the descriptive section:**

**Top 5 Resolution 6 Hexagon Ids:**
['862664c1fffffff',
 '862664cafffffff',
 '862759347ffffff',
 '862664c17ffffff',
 '862664d8fffffff']
 
**Top 5 Resolution 7 Hexagon Ids:** 
['872664c1effffff',
 '872664c1affffff',
 '872664ca9ffffff',
 '872759343ffffff',
 '872664c16ffffff']
 
**Top 5 Census Tract Ids:** 
[17031980000.0, 17031839100.0, 17031320100.0, 17031081500.0, 17031281900.0]


**With this information, we use the top 5 IDs for each spatial resolution as representatives to find the best hyperparameters for all Ids.**

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVR

**Defining a helper function to do a train-test split for time-series data**

In [6]:
def split_dataset(df, area_id):
    # Splitting the dataframe into feature columns and the prediction column
    X = df.loc[area_id].iloc[:, 1:]
    y = df.loc[area_id].iloc[:, 0]

    # Splitting the data into a 70-30 train-test split without shuffle since its time series data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
    return X_train, X_test, y_train, y_test

**Defining a helper function to do RandomizedSearchCV for a given set of area iDs and returning the CV results**

In [17]:
def hyperparameter_tuning_svr(spatialres, timeres, area_id, X_train, y_train, param_grid, n_iter, n_splits=5, random_state=42, n_jobs=8):
    # Loading the time series split CV
    tscv = TimeSeriesSplit(n_splits=n_splits)
    
    # Performing Randomized Search on the predefined parameter grid
    random_search = RandomizedSearchCV(SVR(cache_size = 3000), 
                                       param_grid, 
                                       cv=tscv, 
                                       scoring='neg_mean_squared_error', 
                                       refit=True, 
                                       verbose=0,
                                       n_iter=n_iter, 
                                       random_state=random_state, 
                                       n_jobs=n_jobs)

    # Perform randomized search on the training data
    random_search.fit(X_train, y_train)

    # Get the best SVR model from the random search
    best_svr = random_search.best_estimator_

    # Get the best cross-validation test scores and parameters
    cv_results = random_search.cv_results_
    best_score = random_search.best_score_
    best_params = random_search.best_params_
    
    # Calculate training scores using cross-validation
    train_scores = cross_val_score(best_svr, X_train, y_train, cv=tscv, scoring='neg_mean_squared_error')

    results = {
        'CV Mean MSE': -cv_results['mean_test_score'].mean(),
        'CV Std MSE': cv_results['std_test_score'].mean(), 
        'Mean Training Score': train_scores.min(),
        'Std Training Score': train_scores.std(),
        'CV Best Score': best_score,
        'Best Parameters': best_params,
        'CV Iter.': f"{n_iter} (max. 40)",
        'K Fold' : n_splits,
        'Spatial Res.': spatialres,
        'Time Res.': timeres,
        'Area Id': area_id
        }
    return results



def id_iterator(df, top_5, spatialres, timeres, param_grid, n_iter):   
    # Define column names
    column_names = ['CV Mean MSE', 'CV Std MSE', 'Mean Training Score', 'Std Training Score', 'CV Best Score', 'Best Parameters', 'K Fold', 'CV Iter.', 'Spatial Res.', 'Time Res.', 'Area Id']

    # Initialize an empty DataFrame with column names
    results_df = pd.DataFrame(columns=column_names)
    
    for num in top_5:
        # Train-Test split
        X_train, X_test, y_train, y_test = split_dataset(df, num)
        
        # Searching the best hyperparameter set (only on X_train and y_train for CV, we dont touch the test data)
        results = hyperparameter_tuning_svr(spatialres, timeres, num, X_train, y_train, param_grid, n_iter)

        # Convert the results dictionary into a DataFrame row
        results_row = pd.DataFrame([results], columns=results_df.columns)
        
        # Append the results to the DataFrame
        results_df = pd.concat([results_df, results_row], ignore_index=True)
        
    return results_df


**Searching for the best hyperparameter set for the top 5 Ids of each spatial resolution with a predefined parameter grid**

In [13]:
# Top 5 Hexagon and Census tract Id's
top_5_h3_6 = ['862664c1fffffff', '862664cafffffff', '862759347ffffff', '862664c17ffffff', '862664d8fffffff']
top_5_h3_7 = ['872664c1effffff', '872664c1affffff', '872664ca9ffffff', '872759343ffffff', '872664c16ffffff']
top_5_census = np.array([17031980000.0, 17031839100.0, 17031320100.0, 17031081500.0, 17031281900.0])

# Defining the parameter grid
param_grid = [
    {'C': [1000, 5000, 10000], 'kernel': ['linear']},
    {'C': [1000, 5000, 10000], 'gamma': [0.1, 0.05, 0.01, 0.005, 0.001], 'kernel': ['rbf']},
    {'C': [1000, 5000, 10000], 'degree': [3, 4, 5, 6], 'kernel': ['poly']}
]

##### Census Tract Resolution

In [9]:
# Iterate over the Ids and search for the best hyperparameters over the predefined parameter grid
results_census_24h = id_iterator(taxi_by_census_tract_24H, top_5_census, 'Census', '24H', param_grid, 30)
results_census_24h

,CV Mean MSE,CV Std MSE,Mean Training Score,Std Training Score,CV Best Score,Best Parameters,K Fold,CV Iter.,Spatial Res.,Time Res.,Area Id
0,158238.041472,143087.025152,-292568.283838,102565.587490,-115053.449160,"{'kernel': 'rbf', 'gamma': 0.01, 'C': 1000}",5,30 (max. 40),Census,24H,1.703198e+10
1,39558.897790,40412.374644,-89329.379028,31512.010624,-29190.914662,"{'kernel': 'rbf', 'gamma': 0.1, 'C': 5000}",5,30 (max. 40),Census,24H,1.703184e+10
2,44750.465474,41117.611716,-81546.273985,27882.763275,-33401.177260,"{'kernel': 'rbf', 'gamma': 0.1, 'C': 5000}",5,30 (max. 40),Census,24H,1.703132e+10
3,25808.717497,26465.985203,-50823.672046,17949.119634,-17043.824265,"{'kernel': 'rbf', 'gamma': 0.1, 'C': 10000}",5,30 (max. 40),Census,24H,1.703108e+10
4,17045.307986,12329.121925,-20788.203209,6916.014374,-11407.412076,"{'kernel': 'rbf', 'gamma': 0.05, 'C': 10000}",5,30 (max. 40),Census,24H,1.703128e+10


In [14]:
# Iterate over the Ids and search for the best hyperparameters over the predefined parameter grid
results_census_6h = id_iterator(taxi_by_census_tract_6H, top_5_census, 'Census', '6H', param_grid, 30)
results_census_6h

C:\Users\OK\AppData\Roaming\Python\Python310\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
2 fits failed out of a total of 150.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
2 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\OK\AppData\Roaming\Python\Python310\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\OK\AppData\Roaming\Python\Python310\site-packages\sklearn\svm\_base.py", line 270, in fit
    raise ValueError(
ValueError: The dual coefficients or intercepts are not finite. The input data may contain large values and need to bepreprocessed.

  warnings.warn(s

,CV Mean MSE,CV Std MSE,Mean Training Score,Std Training Score,CV Best Score,Best Parameters,K Fold,CV Iter.,Spatial Res.,Time Res.,Area Id
0,22889.676531,22531.665316,-47094.137351,16529.570145,-16541.805485,"{'kernel': 'rbf', 'gamma': 0.1, 'C': 5000}",5,30 (max. 40),Census,6H,1.703198e+10
1,10050.992314,9683.061017,-21167.673119,7400.490500,-6873.865628,"{'kernel': 'rbf', 'gamma': 0.1, 'C': 1000}",5,30 (max. 40),Census,6H,1.703184e+10
2,NaN,NaN,-15353.342021,5233.832642,-5498.978232,"{'kernel': 'rbf', 'gamma': 0.01, 'C': 10000}",5,30 (max. 40),Census,6H,1.703132e+10
3,4290.655847,4285.493794,-8053.103016,2872.489395,-2593.579624,"{'kernel': 'rbf', 'gamma': 0.01, 'C': 10000}",5,30 (max. 40),Census,6H,1.703108e+10
4,7260.843455,5893.563373,-6068.678958,1806.946594,-3184.354041,"{'kernel': 'rbf', 'gamma': 0.005, 'C': 10000}",5,30 (max. 40),Census,6H,1.703128e+10


In [ ]:
# Iterate over the Ids and search for the best hyperparameters over the predefined parameter grid
results_census_2h = id_iterator(taxi_by_census_tract_2H, top_5_census, 'Census', '2H', param_grid, 20)
results_census_2h

In [ ]:
# Iterate over the Ids and search for the best hyperparameters over the predefined parameter grid
results_census_1h = id_iterator(taxi_by_census_tract_1H, top_5_census, 'Census', '1H', param_grid, 30)
results_census_1h

##### Hexagon Resolution 7

In [24]:
# Iterate over the Ids and search for the best hyperparameters over the predefined parameter grid
results_7_24h = id_iterator(taxi_by_h3_7_24H , top_5_h3_7, 7, '24H', param_grid, 30)
results_7_24h

,CV Mean MSE,CV Std MSE,Mean Training Score,Std Training Score,CV Best Score,Best Parameters,K Fold,CV Iter.,Spatial Res.,Time Res.,Area Id
0,1.006748e+06,814976.492675,-1.695110e+06,553394.324137,-730115.589880,"{'kernel': 'rbf', 'gamma': 0.05, 'C': 1000}",5,30 (max. 40),7,24H,872664c1effffff
1,3.927139e+05,411179.110685,-8.537352e+05,291755.613301,-303993.756495,"{'kernel': 'rbf', 'gamma': 0.1, 'C': 10000}",5,30 (max. 40),7,24H,872664c1affffff
2,6.344815e+04,33118.544205,-7.831657e+04,23570.926974,-44861.893961,"{'kernel': 'rbf', 'gamma': 0.01, 'C': 1000}",5,30 (max. 40),7,24H,872664ca9ffffff
3,4.284880e+05,285034.306721,-5.033181e+05,151766.314610,-275186.932336,"{'kernel': 'rbf', 'gamma': 0.01, 'C': 1000}",5,30 (max. 40),7,24H,872759343ffffff
4,2.600013e+04,19594.016525,-4.129294e+04,12605.561449,-19240.967667,"{'kernel': 'rbf', 'gamma': 0.001, 'C': 1000}",5,30 (max. 40),7,24H,872664c16ffffff


In [25]:
# Iterate over the Ids and search for the best hyperparameters over the predefined parameter grid
results_7_6h = id_iterator(taxi_by_h3_7_6H , top_5_h3_7, 7, '6H', param_grid, 30)
results_7_6h

,CV Mean MSE,CV Std MSE,Mean Training Score,Std Training Score,CV Best Score,Best Parameters,K Fold,CV Iter.,Spatial Res.,Time Res.,Area Id
0,193804.307808,120786.970102,-311928.040895,93161.219931,-145515.511572,"{'kernel': 'rbf', 'gamma': 0.01, 'C': 10000}",5,30 (max. 40),7,6H,872664c1effffff
1,85967.798437,72256.000764,-164108.216194,54163.013185,-61776.240666,"{'kernel': 'rbf', 'gamma': 0.01, 'C': 10000}",5,30 (max. 40),7,6H,872664c1affffff
2,34927.508298,16598.279850,-28549.556568,6363.490037,-18361.326773,"{'kernel': 'rbf', 'gamma': 0.001, 'C': 10000}",5,30 (max. 40),7,6H,872664ca9ffffff
3,81090.337865,56465.798477,-131911.477836,43176.879095,-62979.303531,"{'kernel': 'rbf', 'gamma': 0.01, 'C': 1000}",5,30 (max. 40),7,6H,872759343ffffff
4,6630.104140,3552.053673,-5472.153839,1314.828896,-3101.354287,"{'kernel': 'rbf', 'gamma': 0.01, 'C': 1000}",5,30 (max. 40),7,6H,872664c16ffffff


In [ ]:
# Iterate over the Ids and search for the best hyperparameters over the predefined parameter grid
results_7_2h = id_iterator(taxi_by_h3_7_2H , top_5_h3_7, 7, '2H', param_grid, 30)
results_7_2h

In [ ]:
# Iterate over the Ids and search for the best hyperparameters over the predefined parameter grid
results_7_1h = id_iterator(taxi_by_h3_7_1H , top_5_h3_7, 7, '1H', param_grid, 30)
results_7_1h

##### Hexagon Resolution 6

In [21]:
# Iterate over the Ids and search for the best hyperparameters over the predefined parameter grid
results_6_24h = id_iterator(taxi_by_h3_6_24H , top_5_h3_6, 6, '24H', param_grid, 30)
results_6_24h

,CV Mean MSE,CV Std MSE,Mean Training Score,Std Training Score,CV Best Score,Best Parameters,K Fold,CV Iter.,Spatial Res.,Time Res.,Area Id
0,3.152407e+06,3.119856e+06,-6.338772e+06,2.146716e+06,-2.330928e+06,"{'kernel': 'rbf', 'gamma': 0.1, 'C': 10000}",5,30 (max. 40),6,24H,862664c1fffffff
1,8.317602e+04,4.511012e+04,-1.112428e+05,2.790178e+04,-6.133161e+04,"{'kernel': 'rbf', 'gamma': 0.005, 'C': 1000}",5,30 (max. 40),6,24H,862664cafffffff
2,4.284880e+05,2.850343e+05,-5.033181e+05,1.517663e+05,-2.751869e+05,"{'kernel': 'rbf', 'gamma': 0.01, 'C': 1000}",5,30 (max. 40),6,24H,862759347ffffff
3,5.643053e+04,4.129229e+04,-7.400804e+04,2.429650e+04,-4.418170e+04,"{'kernel': 'rbf', 'gamma': 0.05, 'C': 1000}",5,30 (max. 40),6,24H,862664c17ffffff
4,7.456806e+03,3.005001e+03,-6.832324e+03,1.373012e+03,-5.157932e+03,"{'kernel': 'poly', 'degree': 3, 'C': 1000}",5,30 (max. 40),6,24H,862664d8fffffff


In [22]:
# Iterate over the Ids and search for the best hyperparameters over the predefined parameter grid
results_6_6h = id_iterator(taxi_by_h3_6_6H , top_5_h3_6, 6, '6H', param_grid, 30)
results_6_6h

,CV Mean MSE,CV Std MSE,Mean Training Score,Std Training Score,CV Best Score,Best Parameters,K Fold,CV Iter.,Spatial Res.,Time Res.,Area Id
0,580889.489230,451627.017806,-1.162762e+06,373907.915258,-454299.534940,"{'kernel': 'rbf', 'gamma': 0.05, 'C': 5000}",5,30 (max. 40),6,6H,862664c1fffffff
1,42039.779908,18311.440311,-3.525462e+04,7547.201809,-22973.060432,"{'kernel': 'rbf', 'gamma': 0.001, 'C': 10000}",5,30 (max. 40),6,6H,862664cafffffff
2,81090.338273,56465.798011,-1.319115e+05,43176.879095,-62979.303531,"{'kernel': 'rbf', 'gamma': 0.01, 'C': 1000}",5,30 (max. 40),6,6H,862759347ffffff
3,13670.772544,7233.712782,-1.314687e+04,3131.981758,-7458.643940,"{'kernel': 'rbf', 'gamma': 0.005, 'C': 10000}",5,30 (max. 40),6,6H,862664c17ffffff
4,18319.055024,6649.461567,-8.126637e+03,480.805260,-7520.469741,"{'kernel': 'rbf', 'gamma': 0.001, 'C': 5000}",5,30 (max. 40),6,6H,862664d8fffffff


In [23]:
# Iterate over the Ids and search for the best hyperparameters over the predefined parameter grid
results_6_2h = id_iterator(taxi_by_h3_6_2H , top_5_h3_6, 6, '2H', param_grid, 30)
results_6_2h

,CV Mean MSE,CV Std MSE,Mean Training Score,Std Training Score,CV Best Score,Best Parameters,K Fold,CV Iter.,Spatial Res.,Time Res.,Area Id
0,94114.353830,67919.322925,-175806.973602,57077.185928,-66993.673881,"{'kernel': 'rbf', 'gamma': 0.01, 'C': 1000}",5,30 (max. 40),6,2H,862664c1fffffff
1,9211.494230,4519.607002,-5814.502791,1319.325893,-3593.916321,"{'kernel': 'rbf', 'gamma': 0.001, 'C': 1000}",5,30 (max. 40),6,2H,862664cafffffff
2,12176.322309,8566.706902,-16743.680870,5237.933886,-8777.972525,"{'kernel': 'rbf', 'gamma': 0.001, 'C': 1000}",5,30 (max. 40),6,2H,862759347ffffff
3,3410.477379,1734.454339,-2404.633812,630.109463,-1234.312038,"{'kernel': 'rbf', 'gamma': 0.001, 'C': 1000}",5,30 (max. 40),6,2H,862664c17ffffff
4,4812.412968,2325.459077,-1109.105039,94.605188,-1018.596423,"{'kernel': 'rbf', 'gamma': 0.001, 'C': 1000}",5,30 (max. 40),6,2H,862664d8fffffff


In [26]:
# Iterate over the Ids and search for the best hyperparameters over the predefined parameter grid
results_6_1h = id_iterator(taxi_by_h3_6_1H , top_5_h3_6, 6, '1H', param_grid, 30)
results_6_1h

,CV Mean MSE,CV Std MSE,Mean Training Score,Std Training Score,CV Best Score,Best Parameters,K Fold,CV Iter.,Spatial Res.,Time Res.,Area Id
0,29354.782734,19132.428958,-46949.999364,15285.468450,-17645.803057,"{'kernel': 'rbf', 'gamma': 0.005, 'C': 1000}",5,30 (max. 40),6,1H,862664c1fffffff
1,4157.707418,2596.923817,-1651.492422,377.782798,-972.552540,"{'kernel': 'rbf', 'gamma': 0.001, 'C': 1000}",5,30 (max. 40),6,1H,862664cafffffff
2,4301.754600,3396.658554,-5123.495146,1586.632242,-2470.520501,"{'kernel': 'rbf', 'gamma': 0.001, 'C': 1000}",5,30 (max. 40),6,1H,862759347ffffff
3,1721.704391,1109.230671,-726.040643,198.193463,-348.243485,"{'kernel': 'rbf', 'gamma': 0.001, 'C': 1000}",5,30 (max. 40),6,1H,862664c17ffffff
4,2558.227193,1504.419197,-318.607109,13.846182,-304.849932,"{'kernel': 'rbf', 'gamma': 0.001, 'C': 1000}",5,30 (max. 40),6,1H,862664d8fffffff


**Selection:**
+ **kernel: rbf (Radial Basis Function)**
+ **gamma: 0.05**
+ **C: 5000**

**On the basis of this optimal hyperparameter set we are evaluating all area id's.**

## Model Building & Evaluation

#### Defining helper functions

**Defining non zero evaluation metrics**

In [27]:
def non_zero_mape(y_true, y_pred):
    non_zero_indices = y_true != 0
    y_true_non_zero = y_true[non_zero_indices]
    y_pred_non_zero = y_pred[non_zero_indices]
    
    absolute_percentage_errors = np.abs((y_true_non_zero - y_pred_non_zero) / y_true_non_zero)
    non_zero_mape = np.mean(absolute_percentage_errors) * 100
    
    return non_zero_mape

def non_zero_mse(y_true, y_pred):
    non_zero_indices = y_true != 0
    y_true_non_zero = y_true[non_zero_indices]
    y_pred_non_zero = y_pred[non_zero_indices]
    
    non_zero_mse = np.mean((y_true_non_zero - y_pred_non_zero)**2)
    
    return non_zero_mse

def non_zero_mae(y_true, y_pred):
    non_zero_indices = y_true != 0
    y_true_non_zero = y_true[non_zero_indices]
    y_pred_non_zero = y_pred[non_zero_indices]
    
    non_zero_mae = np.mean(np.abs(y_true_non_zero - y_pred_non_zero))
    
    return non_zero_mae

**Defining helper functions to iterate over a given area id matrix for evaluation and using the predictor with the best parameter set**

In [28]:
def evaluate_id(best_svr ,spatialres, timeres, area_id, X_test, y_test):
    # Evaluate the best SVR model on the test set
    y_test_pred = best_svr.predict(X_test)
    test_mse = np.sqrt(mean_squared_error(y_test, y_test_pred))
    test_mae = mean_absolute_error(y_test, y_test_pred)
    r2 = r2_score(y_true = y_test, y_pred = y_test_pred)
    nz_mape = non_zero_mape(y_test, y_test_pred)
    nz_mse = np.sqrt(non_zero_mse(y_test, y_test_pred))
    nz_mae = non_zero_mae(y_test, y_test_pred)
    
    # Save the resulting metrics
    results = {
        'RMSE': test_mse,
        'MAE': test_mae,
        'Non Zero RMSE': nz_mse,
        'Non Zero MAE': nz_mae,
        'Non Zero MAPE': nz_mape,
        'R-squared': r2,
        'Spatial Res.' : spatialres,
        'Time Res.' : timeres,
        'Id': area_id
    }
    return results


def id_eval_iterator(best_params, spatialres, timeres, area_ids, df):
    # Define column names
    column_names = ['RMSE', 'MAE', 'R-squared', 'Non Zero RMSE', 'Non Zero MAE', 'Non Zero MAPE', 'Spatial Res.', 'Time Res.', 'Id']

    # Initialize an empty DataFrame with column names
    results_df = pd.DataFrame(columns=column_names)
    
    # Create an SVR model with the best parameters
    #best_svr = SVR(**best_params, cache_size = 1000)
    
    # Iterating over all area id's -> fit and evaluate all area id's
    for num in area_ids:
        # Train-Test split
        X_train, X_test, y_train, y_test = split_dataset(df, num)
        
        # Create an SVR model with the best parameters
        best_svr = SVR(**best_params, cache_size = 3000)
        
        # Fit the SVR model to the dataset corresponding to the area id 'num'
        best_svr.fit(X_train, y_train)
        
        # Train and Evaluate the Dataset for a fixed Area id
        results = evaluate_id(best_svr ,spatialres, timeres, num, X_test, y_test)
        
        # Convert the results dictionary into a DataFrame row
        results_row = pd.DataFrame([results], columns=results_df.columns)
        
        # Append the results to the DataFrame
        results_df = pd.concat([results_df, results_row], ignore_index=True)
        
    return results_df

#### Best Parameter Model Evaluation

**Getting all area id's of all spatio-temporal resolutions**

In [29]:
# Hexagon resolution 6 Id's 
h3_6_1H_ids = taxi_by_h3_6_1H.index.get_level_values('h3_6_pickup').unique()
h3_6_2H_ids = taxi_by_h3_6_2H.index.get_level_values('h3_6_pickup').unique()
h3_6_6H_ids = taxi_by_h3_6_6H.index.get_level_values('h3_6_pickup').unique()
h3_6_24H_ids = taxi_by_h3_6_24H.index.get_level_values('h3_6_pickup').unique()

# Hexagon resolution 7 Id's 
h3_7_1H_ids = taxi_by_h3_7_1H.index.get_level_values('h3_7_pickup').unique()
h3_7_2H_ids = taxi_by_h3_7_2H.index.get_level_values('h3_7_pickup').unique()
h3_7_6H_ids = taxi_by_h3_7_6H.index.get_level_values('h3_7_pickup').unique()
h3_7_24H_ids = taxi_by_h3_7_24H.index.get_level_values('h3_7_pickup').unique()

# Census tract Id's 
census_tract_1H_ids = taxi_by_census_tract_1H.index.get_level_values('pickup_census_tract').unique()
census_tract_2H_ids = taxi_by_census_tract_2H.index.get_level_values('pickup_census_tract').unique()
census_tract_6H_ids = taxi_by_census_tract_6H.index.get_level_values('pickup_census_tract').unique()
census_tract_24H_ids = taxi_by_census_tract_24H.index.get_level_values('pickup_census_tract').unique()

**Iterating over all area IDs for each spatio-temporal resolution and evaluating the predictive perfomance for each ID**

In [30]:
# Best parameter set
best_params = {'kernel': 'rbf', 'gamma': 0.05, 'C': 5000}

##### Census Tract Resolution

In [36]:
# Iterate over the Ids in the dataset and evaluate the best NN model
eval_census_24H = id_eval_iterator(best_params, 'Census', '24H', census_tract_24H_ids, taxi_by_census_tract_24H)
eval_census_24H.describe()

,RMSE,MAE,R-squared,Non Zero RMSE,Non Zero MAE,Non Zero MAPE,Id
count,483.000000,483.000000,483.000000,321.000000,321.000000,321.000000,4.830000e+02
mean,8.436359,6.626036,-2.897146,13.727910,11.124773,99.050980,1.703138e+10
std,45.744181,36.920442,8.419324,55.523829,44.763911,33.858630,2.997544e+05
min,0.054577,0.013699,-125.234653,0.008154,0.008154,0.815408,1.703101e+10
25%,0.278701,0.172101,-2.210349,1.360413,1.202459,81.678795,1.703108e+10
50%,0.500742,0.351356,-0.442338,2.016910,1.970033,100.000000,1.703132e+10
75%,0.978499,0.710019,0.000000,3.351860,3.000321,106.548885,1.703169e+10
max,558.911108,453.443123,0.237710,558.911108,453.443123,318.102195,1.703198e+10


In [37]:
# Iterate over the Ids in the dataset and evaluate the best SVR model
eval_census_6H = id_eval_iterator(best_params, 'Census', '6H', census_tract_6H_ids, taxi_by_census_tract_6H)
eval_census_6H.describe()

,RMSE,MAE,R-squared,Non Zero RMSE,Non Zero MAE,Non Zero MAPE,Id
count,483.000000,483.000000,483.000000,321.000000,321.000000,321.000000,4.830000e+02
mean,3.787701,2.645533,-1.858349,7.630206,5.997951,103.406277,1.703138e+10
std,20.426266,15.009748,4.996642,25.875613,19.465021,24.401415,2.997544e+05
min,0.044783,0.003425,-47.872503,0.537003,0.537003,53.700270,1.703101e+10
25%,0.135337,0.084261,-1.348114,1.368420,1.248948,94.588754,1.703108e+10
50%,0.219635,0.137540,-0.271354,2.000000,1.928894,100.000000,1.703132e+10
75%,0.393381,0.253830,0.000000,2.961223,2.798259,107.082156,1.703169e+10
max,241.950292,184.132807,0.016517,241.950292,184.132807,294.997323,1.703198e+10


In [ ]:
# Iterate over the Ids in the dataset and evaluate the best SVR model
eval_census_2H = id_eval_iterator(best_params, 'Census', '2H', census_tract_2H_ids, taxi_by_census_tract_2H)
eval_census_2H.describe()

In [ ]:
# Iterate over the Ids in the dataset and evaluate the best SVR model
eval_census_1H = id_eval_iterator(best_params, 'Census', '1H', census_tract_1H_ids, taxi_by_census_tract_1H)
eval_census_1H.describe()

##### Hexagon Resolution 7

In [34]:
# Iterate over the Ids in the dataset and evaluate the best SVR model
eval_7_24H = id_eval_iterator(best_params, 7, '24H', h3_7_24H_ids, taxi_by_h3_7_24H)
eval_7_24H.describe()

,RMSE,MAE,R-squared,Non Zero RMSE,Non Zero MAE,Non Zero MAPE
count,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000
mean,87.476060,73.093436,-1.183285,87.474053,73.089898,41.269217
std,290.999720,250.884368,1.050162,291.000295,250.885367,27.963581
min,2.587939,1.990895,-4.791734,2.708429,2.051045,14.537081
25%,5.851872,4.763395,-1.760764,5.851872,4.763395,24.205115
50%,11.062612,8.794054,-0.981212,11.062612,8.794054,31.980487
75%,19.839679,16.126627,-0.462468,19.839679,16.126627,45.437619
max,1849.138942,1638.955411,0.407104,1849.138942,1638.955411,177.991838


In [35]:
# Iterate over the Ids in the dataset and evaluate the best SVR model
eval_7_6H = id_eval_iterator(best_params, 7, '6H', h3_7_6H_ids, taxi_by_h3_7_6H)
eval_7_6H.describe()

,RMSE,MAE,R-squared,Non Zero RMSE,Non Zero MAE,Non Zero MAPE
count,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000
mean,44.169682,34.479976,-1.531202,44.290896,34.607690,115.332925
std,120.521657,95.254557,0.710618,120.486441,95.217984,27.659135
min,1.491623,1.152702,-3.306553,1.577565,1.231216,70.275020
25%,4.990529,3.775373,-1.932313,5.067392,3.769549,97.085179
50%,7.950968,6.370282,-1.347595,8.012565,6.525504,113.959291
75%,22.357625,17.981814,-0.965733,22.488068,18.108438,129.256224
max,725.374585,586.288043,-0.291774,725.374585,586.288043,220.909921


In [ ]:
# Iterate over the Ids in the dataset and evaluate the best SVR model
eval_7_2H = id_eval_iterator(best_params, 7, '2H', h3_7_2H_ids, taxi_by_h3_7_2H)
eval_7_2H.describe()

In [ ]:
# Iterate over the Ids in the dataset and evaluate the best SVR model
eval_7_1H = id_eval_iterator(best_params, 7, '1H', h3_7_1H_ids, taxi_by_h3_7_1H)
eval_7_1H.describe()

##### Hexagon Resolution 6

In [32]:
# Iterate over the Ids in the dataset and evaluate the best SVR model
eval_6_24H = id_eval_iterator(best_params, 6, '24H', h3_6_24H_ids, taxi_by_h3_6_24H)
eval_6_24H.describe()

,RMSE,MAE,R-squared,Non Zero RMSE,Non Zero MAE,Non Zero MAPE
count,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000
mean,241.321222,207.823880,-0.885714,241.326042,207.826286,27.162021
std,712.680571,625.181194,0.970504,712.678889,625.180369,13.721625
min,2.587939,1.990895,-3.254236,2.708429,2.051045,13.169500
25%,12.635276,9.645029,-1.335838,12.635276,9.645029,18.560676
50%,23.946835,20.722714,-0.974022,23.946835,20.722714,21.105198
75%,74.405990,64.220248,0.041802,74.405990,64.220248,31.701157
max,3501.011170,3080.770656,0.256986,3501.011170,3080.770656,72.371601


In [33]:
# Iterate over the Ids in the dataset and evaluate the best SVR model
eval_6_6H = id_eval_iterator(best_params, 6, '6H', h3_6_6H_ids, taxi_by_h3_6_6H)
eval_6_6H.describe()

,RMSE,MAE,R-squared,Non Zero RMSE,Non Zero MAE,Non Zero MAPE
count,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000
mean,119.419382,95.103336,-1.471994,119.515664,95.206715,106.621770
std,279.145998,222.136783,0.684151,279.107665,222.095997,29.122964
min,1.491623,1.152702,-3.301198,1.901729,1.360939,70.275020
25%,9.290984,7.301085,-1.662192,9.326671,7.405233,79.464638
50%,23.469379,18.321380,-1.256533,23.469379,18.321380,99.859338
75%,78.839827,61.926555,-1.039123,78.839827,61.926555,123.557009
max,1362.607240,1086.028925,-0.620953,1362.607240,1086.028925,192.073577


In [ ]:
# Iterate over the Ids in the dataset and evaluate the best SVR model
eval_6_2H = id_eval_iterator(best_params, 6, '2H', h3_6_2H_ids, taxi_by_h3_6_2H)
eval_6_2H.describe()

In [ ]:
# Iterate over the Ids in the dataset and evaluate the best SVR model
eval_6_1H = id_eval_iterator(best_params, 6, '1H', h3_6_1H_ids, taxi_by_h3_6_1H)
eval_6_1H.describe()

#### Saving the Evaluation Results for Comparison

In [38]:
#summary_svm_census_24H = eval_census_24H.describe()
#summary_svm_census_6H = eval_census_6H.describe()
#summary_svm_census_2H = eval_census_2H.describe()
#summary_svm_census_1H = eval_census_1H.describe()
#summary_svm_7_24H = eval_7_24H.describe()
#summary_svm_7_6H = eval_7_6H.describe()
#summary_svm_7_2H = eval_7_2H.describe()
#summary_svm_7_1H = eval_7_1H.describe()
#summary_svm_6_24H = eval_6_24H.describe()
#summary_svm_6_6H = eval_6_6H.describe()
#summary_svm_6_2H = eval_6_2H.describe()
#summary_svm_6_1H = eval_6_1H.describe()


file_path = "./data/datasets/"
#summary_svm_census_24H.to_pickle(f"{file_path}summary_svm_census_24H.pkl")
#summary_svm_census_6H.to_pickle(f"{file_path}summary_svm_census_6H.pkl")
#summary_svm_census_2H.to_pickle(f"{file_path}summary_svm_census_2H.pkl")
#summary_svm_census_1H.to_pickle(f"{file_path}summary_svm_census_1H.pkl")
#summary_svm_7_24H.to_pickle(f"{file_path}summary_svm_7_24H.pkl")
#summary_svm_7_6H.to_pickle(f"{file_path}summary_svm_7_6H.pkl")
#summary_svm_7_2H.to_pickle(f"{file_path}summary_svm_7_2H.pkl")
#summary_svm_7_1H.to_pickle(f"{file_path}summary_svm_7_1H.pkl")
#summary_svm_6_24H.to_pickle(f"{file_path}summary_svm_6_24H.pkl")
#summary_svm_6_6H.to_pickle(f"{file_path}summary_svm_6_6H.pkl")
#summary_svm_6_2H.to_pickle(f"{file_path}summary_svm_6_2H.pkl")
#summary_svm_6_1H.to_pickle(f"{file_path}summary_svm_6_1H.pkl")

## Outlook

+ How could the model be improved further? Explain some of the improvement levers that you might focus on in a follow-up project.


**Shortfalls & Improvements:**

+ Due to out current computational resources, we are only able to compute the best hyperparameters for a small set of ids for each spatial resolution. An improvement would be to expand hyperparameter search to include a wider range of area IDs, going beyond top-demand regions.
+ Currently, our capacity only allows for a limited random search within a small parameter grid. With additional resources, we could expand the hyperparameter set, leading to better evaluation metrics across the majority of areas. For instance, we could select the optimal hyperparameter set for each specific spatial resolution, rather than settling for a one-size-fits-all compromise after the hyperparameter search.
+ Furthermore, the model's predictive capabilities are constrained by the current feature set. To enhance this, investing more time in comprehensive feature engineering could be valuable. Incorporating additional attributes, like 'distance to next station', 'station count', 'urban fabric continuity', 'industrial and green urban areas', 'agricultural zones', 'river crossings', 'proximity to Central Station', and 'distance to University', would aid in distinguishing low and high demand areas. This would bolster predictions, especially in regions with sparse data.
+ In numerous census tracts, data scarcity is a common issue, and only a small number of tracts see significant demand. Expanding the dataset often leads to better model performance, especially because many areas have a lot of zeros and frequently lack trip data. This sparsity makes it difficult to make accurate predictions.

